In [ ]:
## GNB.py - process GNB Widgets, Points, Net Auth Volume

In [ ]:
from pyspark.context import SparkContext
from pyspark.sql import HiveContext, SQLContext,SparkSession
from pyspark.sql import functions as F
from datetime import date,timedelta
from datetime import datetime
sc = SparkContext(appName = "Combined_Table")
sql_context = HiveContext(sc)
spark = SparkSession(sc)
spark = (SparkSession.builder.enableHiveSupport().getOrCreate())

In [ ]:
spark 
import pandas as pd
import os
import numpy as np
import datetime
import time
import glob
import os
from numpy import nan
from pyspark.sql import functions as F
from pyspark.sql.functions import format_string

In [ ]:
exec(open('/home/thural2/THURAL2/thural2/record.py').read())


In [ ]:
rec_date

In [ ]:
################################# START GNB PROCESSING ###########################################

df12=spark.sql("select * from anp_cabbtdct1_working.SBB_GNB")
gnb=df12.toPandas()

## make all columns lowercase 
gnb.columns=gnb.columns.str.lower()

## get only this year's data
## LATEST DATA
gnb_widgets1_filter=gnb[(gnb.inthis>=m1)]

from datetime import datetime
########## get numeric month
gnb_widgets1_filter['month_dt'] = pd.to_datetime(gnb_widgets1_filter.inthis, format='%Y%m')
#gnb_widgets1_filter['month_dt'] = gnb_widgets1_filter['month_dt'].dt.month
gnb_widgets1_filter['month_dt'] = gnb_widgets1_filter['month_dt'].dt.strftime("%B").str.lower()

#import calendar
#gnb_widgets1_filter['month_dt'] = gnb_widgets1_filter['month_dt'].apply(lambda x: calendar.month_name[x]).str.lower()
#print(gnb_widgets1_filter.month_dt.unique())

##rename logon_id
gnb_widgets1_filter.rename(columns = {"logon_id":'acf2_id'}, inplace = True) 
 
## create record_date
gnb_widgets1_filter['record_date']=rec_date

## get relevant variables
gnb_wid00=gnb_widgets1_filter[['acf2_id','slsbran','prodcd','salebal','franchise_cust',
                              'professional_cust','month_dt','gnb','auth_credit','gnb_volume',
                              'gnb_points','job_code','am_cost_center','region_id','record_date']]
    
## convert to float
gnb_wid00 = gnb_wid00.astype({'gnb_volume': float})
gnb_wid00 = gnb_wid00.astype({'gnb_points': float})
gnb_wid00 = gnb_wid00.astype({'auth_credit': float})

gnb_wid1=gnb_wid00[(gnb_wid00.job_code=='251801')] 


In [ ]:
gnb_wid00.info()

In [ ]:
gnb_wid00.head(5)

In [ ]:
##############################################################################################################
###################################### GNB WIDGETS (SBB FILTER) ##############################################

gnb_widgets2_filter=gnb_wid1[(gnb_wid1.gnb=='Y') & (gnb_wid1.gnb_volume>0)]

## create a variable to identify the metric
gnb_widgets3_filter=gnb_widgets2_filter.assign(metric_name='Total GNB Widgets (Actual)')
gnb_wid_df9=gnb_widgets3_filter[['metric_name','acf2_id','slsbran','prodcd','salebal','franchise_cust',
                              'professional_cust','month_dt','gnb','auth_credit','gnb_volume',
                              'gnb_points','job_code','am_cost_center','region_id','record_date']]

## group by logon_id, metric_name, month_dt
gnb_wid_df1=gnb_wid_df9.groupby(['acf2_id','month_dt','metric_name','record_date'],as_index=False).aggregate({
                'gnb':'count'  # count widgets
                })

##transpose data
gnb_wid_df2 = gnb_wid_df1.pivot_table(index=['acf2_id','metric_name','record_date'], columns=['month_dt'],
                     values='gnb', aggfunc='first', fill_value=0).reset_index()

#### get months
##rename columns **************CHANGE FOR FUTURE
cols = ['november','december','january','february','march','april','may','june','july',
        'august','september','october']
net_a = pd.DataFrame(list, index=['a',], columns=cols)

complementary = [c for c in net_a if c not in gnb_wid_df2]
gnb_wid_df3= pd.concat([net_a[complementary], gnb_wid_df2], axis=1, join='outer')

##delete the extra filler row
gnb_wid_df3= gnb_wid_df3.drop(labels='a', axis=0)
##convert to float
gnb_wid_df3 = gnb_wid_df3.astype({"november": float})
gnb_wid_df3 = gnb_wid_df3.astype({"december": float})
gnb_wid_df3 = gnb_wid_df3.astype({"january": float})
gnb_wid_df3 = gnb_wid_df3.astype({"february": float})
gnb_wid_df3 = gnb_wid_df3.astype({"march": float})
gnb_wid_df3 = gnb_wid_df3.astype({"april": float})
gnb_wid_df3 = gnb_wid_df3.astype({"may": float})
gnb_wid_df3 = gnb_wid_df3.astype({"june": float})
gnb_wid_df3 = gnb_wid_df3.astype({"july": float})
gnb_wid_df3 = gnb_wid_df3.astype({"august": float})
gnb_wid_df3 = gnb_wid_df3.astype({"september": float})
gnb_wid_df3 = gnb_wid_df3.astype({"october": float})

##order the vars
gnb_wid_df3=gnb_wid_df3[['metric_name','acf2_id','november','december','january',
           'february','march','april','may','june','july','august','september','october','record_date']]


In [ ]:
############################################################################################################
###################################### GNB POINTS (SBB FILTER) ##############################################

df_gnb=gnb_wid1[(gnb_wid1.gnb_points>0)]

##filter out the exclusion codes (including Professional Student LOC and Health LOC)
df_gnb=df_gnb[~df_gnb['prodcd'].isin(professional_tuple)]

###group all data by points and month
df_gnb['metric_name'] = df_gnb['gnb_points']
gnb_imp0= df_gnb.astype({"metric_name": float})
gnb_imp0= gnb_imp0.astype({"metric_name": str})

gnb_imp0["metric_name"].replace({"1.0": "GNB Points 1PT", "4.0": "GNB Points 4PTS","10.0": "GNB Points 10PTS","12.0": "GNB Points 12PTS"}, inplace=True)

gnb_imp1=gnb_imp0.groupby(['acf2_id','month_dt','metric_name','record_date'],as_index=False).aggregate({
    'gnb_points':"sum"  # Sum points
    })

##transpose data
res = gnb_imp1.pivot_table(index=['acf2_id','metric_name','record_date'], columns=['month_dt'],
                          values='gnb_points', aggfunc='first', fill_value=0).reset_index()


############################# create the total gnb points header ############################

result= gnb_imp0.groupby(by=['acf2_id','month_dt','record_date'],as_index=False).aggregate({
    'gnb_points':"sum"  # Sum auth_credit volume
    })

result.rename(columns = {"gnb_points":'SBB TOTAL POINTS'}, inplace = True) 
result['metric_name']='Total GNB Points'


##transpose the data
res1 = result.pivot_table(index=['acf2_id','metric_name','record_date'], columns=['month_dt'],
                          values='SBB TOTAL POINTS', aggfunc='first', fill_value=0).reset_index()

##merge to res
gnb_new1=pd.concat([res,res1],axis=0, ignore_index=True)

#### get months
##rename columns **************CHANGE FOR FUTURE
cols = ['november','december','january','february','march','april','may','june','july',
        'august','september','october']
net_a = pd.DataFrame(list, index=['a',], columns=cols)


complementary = [c for c in net_a if c not in gnb_new1]
gnb_wid_df5= pd.concat([net_a[complementary], gnb_new1], axis=1, join='outer')

##delete the extra filler row
gnb_wid_df5= gnb_wid_df5.drop(labels='a', axis=0)
##convert to float
gnb_wid_df5 = gnb_wid_df5.astype({"november": float})
gnb_wid_df5 = gnb_wid_df5.astype({"december": float})
gnb_wid_df5 = gnb_wid_df5.astype({"january": float})
gnb_wid_df5 = gnb_wid_df5.astype({"february": float})
gnb_wid_df5 = gnb_wid_df5.astype({"march": float})
gnb_wid_df5 = gnb_wid_df5.astype({"april": float})
gnb_wid_df5 = gnb_wid_df5.astype({"may": float})
gnb_wid_df5 = gnb_wid_df5.astype({"june": float})
gnb_wid_df5 = gnb_wid_df5.astype({"july": float})
gnb_wid_df5 = gnb_wid_df5.astype({"august": float})
gnb_wid_df5 = gnb_wid_df5.astype({"september": float})
gnb_wid_df5 = gnb_wid_df5.astype({"october": float})

##order the vars
gnb_wid_df6=gnb_wid_df5[['metric_name','acf2_id','november','december','january',
           'february','march','april','may','june','july','august','september','october','record_date']]


In [ ]:
res.info()

In [ ]:
gnb_wid_df6.head(5)

In [ ]:
#############################################################################################################
############################ GNB NET AUTHORIZED VOLUME (SBB FILTER) #########################################

##TOTAL SBB NET AUTHORIZED VOLUME (SUM OF LON AND LOC)
total_net_auth_filter=gnb_wid1[gnb_wid1.prodcd.isin(gross_all_tuple)]
## create a variable to identify the metric
total_net_auth_filter=total_net_auth_filter.assign(metric_name='Total SBB Net Authorized Volume')
total_net_auth_filter=gnbfilterfunc(total_net_auth_filter)


#######LON
lon_filter=gnb_wid1[gnb_wid1.prodcd.isin(gross_lon_tuple)]
## create a variable to identify the metric
lon_filter=lon_filter.assign(metric_name='SBB Net Authorized LON Volume')
lon_filter=gnbfilterfunc(lon_filter)


#######LOC
loc_filter=gnb_wid1[gnb_wid1.prodcd.isin(gross_loc_tuple)]
## create a variable to identify the metric
loc_filter=loc_filter.assign(metric_name='SBB Net Authorized LOC Volume')
loc_filter=gnbfilterfunc(loc_filter)


#######SBB PROFESSIONAL BANKING
sbb_prof_filter0 = gnb_wid1[(gnb_wid1['professional_cust']=='Y')]
sbb_prof_filter=sbb_prof_filter0[~sbb_prof_filter0['prodcd'].isin(gross_student_loc_tuple)]
## create a variable to identify the metric
sbb_prof_filter=sbb_prof_filter.assign(metric_name='Net SBB Professional Banking')
sbb_prof_filter=gnbfilterfunc(sbb_prof_filter)

#######PROFESSIONAL STUDENT LOC
prof_sloc_filter=gnb_wid1[gnb_wid1.prodcd.isin(gross_student_loc_tuple)]
## create a variable to identify the metric
prof_sloc_filter=prof_sloc_filter.assign(metric_name='Net Professional Student LOC')
prof_sloc_filter=gnbfilterfunc(prof_sloc_filter)

#######FRANCHISE
franchise_filter = gnb_wid1[(gnb_wid1['franchise_cust']=='Y')]
## create a variable to identify the metric
franchise_filter=franchise_filter.assign(metric_name='Net Franchise Banking Net Volume')
franchise_filter=gnbfilterfunc(franchise_filter)


#######TOTAL PROFESSIONAL BANKING NET VOLUME (SUM OF SBB PROF BANKING, PROF STUDENT LOC, HEALTH)
total_prof_filter=pd.concat([sbb_prof_filter,prof_sloc_filter],axis=0)
total_prof_filter=total_prof_filter.drop(columns='metric_name')
## create a variable to identify the metric
total_prof_filter=total_prof_filter.assign(metric_name='Net Professional Banking Net Volume')
total_prof_filter=gnbfilterfunc(total_prof_filter)


########PROFESSIONAL AND FRANCHISE SUM
prof_and_franchise_filter=pd.concat([total_prof_filter,franchise_filter],axis=0)
prof_and_franchise_filter=prof_and_franchise_filter.drop(columns='metric_name')
## create a variable to identify the metric
prof_and_franchise_filter=prof_and_franchise_filter.assign(metric_name='Net Professional and Franchise Banking')
prof_and_franchise_filter=gnbfilterfunc(prof_and_franchise_filter)


## append all of the filters
new_df=pd.concat([total_net_auth_filter,lon_filter,loc_filter,sbb_prof_filter,prof_sloc_filter,
                  franchise_filter,total_prof_filter,prof_and_franchise_filter],axis=0)


In [ ]:
############## AGGREGATE DATA #######################

###group all data by LOGON_ID,MONTH,AND METRIC
    ## rollup to LOGON_ID
new_df1=new_df.groupby(['acf2_id','month_dt','metric_name','record_date'],as_index=False).aggregate({
    'auth_credit':"sum"  # Sum auth_credit volume
    })

##transpose data
new_df2 = new_df1.pivot_table(index=['acf2_id','metric_name','record_date'], columns=['month_dt'],
                     values='auth_credit', aggfunc='first', fill_value=0).reset_index()

#### get months
##rename columns **************CHANGE FOR FUTURE
cols = ['november','december','january','february','march','april','may','june','july',
        'august','september','october']
net_a = pd.DataFrame(list, index=['a',], columns=cols)

complementary = [c for c in net_a if c not in new_df2]
gnb_wid_df7= pd.concat([net_a[complementary], new_df2], axis=1, join='outer')

##delete the extra filler row
gnb_wid_df7= gnb_wid_df7.drop(labels='a', axis=0)
##convert to float
gnb_wid_df7 = gnb_wid_df7.astype({"november": float})
gnb_wid_df7 = gnb_wid_df7.astype({"december": float})
gnb_wid_df7 = gnb_wid_df7.astype({"january": float})
gnb_wid_df7 = gnb_wid_df7.astype({"february": float})
gnb_wid_df7 = gnb_wid_df7.astype({"march": float})
gnb_wid_df7 = gnb_wid_df7.astype({"april": float})
gnb_wid_df7 = gnb_wid_df7.astype({"may": float})
gnb_wid_df7 = gnb_wid_df7.astype({"june": float})
gnb_wid_df7 = gnb_wid_df7.astype({"july": float})
gnb_wid_df7 = gnb_wid_df7.astype({"august": float})
gnb_wid_df7 = gnb_wid_df7.astype({"september": float})
gnb_wid_df7 = gnb_wid_df7.astype({"october": float})

##order the vars
gnb_wid_df8=gnb_wid_df7[['metric_name','acf2_id','november','december','january',
           'february','march','april','may','june','july','august','september','october','record_date']]


In [ ]:
####################### APPEND THE DATAFRAMES FOR WIDGETS,POINTS,VOLUME############################
gnb_all=pd.concat([gnb_wid_df3,gnb_wid_df6,gnb_wid_df8],axis=0)


In [ ]:
gnb_all.metric_name.unique()

In [ ]:
################################### PROCESS FOR AMSB #############################################
newvarfunc(gnb_all)

############## merge to hr/alignment to get am/sm cc
df=spark.sql("select acf2_id,employee_name,am_cost_center,am_cost_center_name,am_cost_center_full_name, \
                     sm_cost_center,sm_cost_center_name from \
                     (select *, row_number() over(partition by acf2_id order by record_date desc) as ran \
                     from anp_cabbtdct1_final.HRM_SBB_ALIGNMENT0)b where ran=1")
dfhr=df.toPandas()

dfhr=dfhr[['acf2_id','employee_name','am_cost_center','am_cost_center_name','am_cost_center_full_name','sm_cost_center','sm_cost_center_name']]

## merge 
gnb_all00=pd.merge(gnb_all,dfhr,how='left',left_on=['acf2_id'],right_on=['acf2_id'])

## replace 0 with NaN so median only counts non-zeros
gnb_all00=gnb_all00.replace(0.0, np.nan)

### calculate qtd and ytd benchmarks
##CREATE A BENCHMARK (based on quarter median)
df_amsb_medianq1=gnb_all00.groupby(['metric_name','sm_cost_center'], as_index=False)['q1'].median()
df_amsb_medianq2=gnb_all00.groupby(['metric_name','sm_cost_center'], as_index=False)['q2'].median()
df_amsb_medianq3=gnb_all00.groupby(['metric_name','sm_cost_center'], as_index=False)['q3'].median()
df_amsb_medianq4=gnb_all00.groupby(['metric_name','sm_cost_center'], as_index=False)['q4'].median()
df_amsb_medianytd=gnb_all00.groupby(['metric_name','sm_cost_center'], as_index=False)['ytd'].median()
##rename median
df_amsb_medianq1.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
df_amsb_medianq2.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
df_amsb_medianq3.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
df_amsb_medianq4.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
df_amsb_medianytd.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)

### merge to dataframe
gnb_pt1=pd.merge(pd.merge(gnb_all00,df_amsb_medianq1,
                on=['metric_name','sm_cost_center'],
                how='left'),df_amsb_medianq2,on=['metric_name','sm_cost_center'],how='left')

gnb_pt2=pd.merge(pd.merge(gnb_pt1,df_amsb_medianq3,
                on=['metric_name','sm_cost_center'],
                how='left'),df_amsb_medianq4,on=['metric_name','sm_cost_center'],how='left')

gnb_pt3=pd.merge(gnb_pt2,df_amsb_medianytd,on=['metric_name','sm_cost_center'],how="left")

gnb_pt3['level']='AMSB'
gnb_pt3['scorecard_filter']='SBB'

gnb_amsb=gnb_pt3[['metric_name','level','acf2_id','employee_name',
                'november','december','january','february','march','april','may','june','july',
                'august','september','october','q1','q1_benchmark','q2','q2_benchmark','q3',
               'q3_benchmark','q4','q4_benchmark','ytd','ytd_benchmark',
               'am_cost_center','am_cost_center_name','am_cost_center_full_name',
               'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


In [ ]:
gnb_amsb.head(5)

In [ ]:
############################# CREATE THE AM SBB OUTPUT ##########################################

gnb110=amgrpfunc(gnb_amsb)

##CREATE THE Q1,Q2,Q3,Q3,YTD VARIABLES
newvarfunc(gnb110)

## replace 0 with NaN so median only counts non-zeros
gnb110=gnb110.replace(0.0, np.nan)

### calculate qtd and ytd benchmarks
##CREATE A BENCHMARK (based on quarter median)
df_am_medianq1=gnb110.groupby(['metric_name'], as_index=False)['q1'].median()
df_am_medianq2=gnb110.groupby(['metric_name'], as_index=False)['q2'].median()
df_am_medianq3=gnb110.groupby(['metric_name'], as_index=False)['q3'].median()
df_am_medianq4=gnb110.groupby(['metric_name'], as_index=False)['q4'].median()
df_am_medianytd=gnb110.groupby(['metric_name'], as_index=False)['ytd'].median()
##rename median
df_am_medianq1.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
df_am_medianq2.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
df_am_medianq3.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
df_am_medianq4.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
df_am_medianytd.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)

### merge to dataframe
gnb_pt11=pd.merge(pd.merge(gnb110,df_am_medianq1,
                on=['metric_name'],
                how='left'),df_am_medianq2,on=['metric_name'],how='left')

gnb_pt22=pd.merge(pd.merge(gnb_pt11,df_am_medianq3,
                on=['metric_name'],
                how='left'),df_am_medianq4,on=['metric_name'],how='left')

gnb_pt33=pd.merge(gnb_pt22,df_am_medianytd,on=['metric_name'],how="left")

gnb_pt33['acf2_id']=''
gnb_pt33['level']='AM'
gnb_pt33['scorecard_filter']='SBB'
gnb_pt33['employee_name']=gnb_pt33.am_cost_center_full_name

gnb_am_sbb=gnb_pt33[['metric_name','level','acf2_id','employee_name','november','december','january',
           'february','march','april','may','june','july','august','september','october',
           'q1','q1_benchmark','q2','q2_benchmark','q3','q3_benchmark','q4','q4_benchmark',
           'ytd','ytd_benchmark','am_cost_center','am_cost_center_name',
           'am_cost_center_full_name','sm_cost_center','sm_cost_center_name',
           'scorecard_filter','record_date']]


In [ ]:
################################ CREATE THE SM SBB OUTPUT ##########################################

gnb111=smgrpfunc(gnb_am_sbb)

##CREATE THE Q1,Q2,Q3,Q3,YTD VARIABLES
newvarfunc(gnb111)

## replace 0 with NaN so median only counts non-zeros
gnb111=gnb111.replace(0.0, np.nan)

### calculate qtd and ytd benchmarks
##CREATE A BENCHMARK (based on quarter median)
df_sm_medianq1=gnb111.groupby(['metric_name'], as_index=False)['q1'].median()
df_sm_medianq2=gnb111.groupby(['metric_name'], as_index=False)['q2'].median()
df_sm_medianq3=gnb111.groupby(['metric_name'], as_index=False)['q3'].median()
df_sm_medianq4=gnb111.groupby(['metric_name'], as_index=False)['q4'].median()
df_sm_medianytd=gnb111.groupby(['metric_name'], as_index=False)['ytd'].median()
##rename median
df_sm_medianq1.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
df_sm_medianq2.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
df_sm_medianq3.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
df_sm_medianq4.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
df_sm_medianytd.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)

### merge to dataframe
gnb_pt111=pd.merge(pd.merge(gnb111,df_sm_medianq1,
                on=['metric_name'],
                how='left'),df_sm_medianq2,on=['metric_name'],how='left')

gnb_pt222=pd.merge(pd.merge(gnb_pt111,df_sm_medianq3,
                on=['metric_name'],
                how='left'),df_sm_medianq4,on=['metric_name'],how='left')

gnb_pt333=pd.merge(gnb_pt222,df_sm_medianytd,on=['metric_name'],how="left")

gnb_pt333['acf2_id']=''
gnb_pt333['level']='SM'
gnb_pt333['scorecard_filter']='SBB'
gnb_pt333['employee_name']=gnb_pt333.sm_cost_center_name
gnb_pt333['am_cost_center']=''
gnb_pt333['am_cost_center_name']=''
gnb_pt333['am_cost_center_full_name']=''

gnb_sm_sbb=gnb_pt333[['metric_name','level','acf2_id','employee_name','november','december','january',
           'february','march','april','may','june','july','august','september','october',
           'q1','q1_benchmark','q2','q2_benchmark','q3','q3_benchmark','q4','q4_benchmark',
           'ytd','ytd_benchmark','am_cost_center','am_cost_center_name',
           'am_cost_center_full_name','sm_cost_center','sm_cost_center_name',
           'scorecard_filter','record_date']]


In [ ]:
##################################################################################################
################################# GET RETAIL FILTER ##############################################

gnb_wid99=gnb_wid00[(gnb_wid00.job_code!='251801') & (gnb_wid00.job_code !='436201')]

gnb_wid99.loc[gnb_wid99['slsbran'] =='3838', "am_cost_center"] ='3838'
gnb_wid99.loc[gnb_wid99['slsbran'] =='5311', "am_cost_center"] ='5311'


In [ ]:
####################################################################################################
##################################### RETAIL GNB WIDGETS############################################

gnb_widgets2_filter=gnb_wid99[(gnb_wid99.gnb=='Y') & (gnb_wid99.gnb_volume>0)]

## create a variable to identify the metric
gnb_widgets3_filter=gnb_widgets2_filter.assign(metric_name='Total GNB Widgets (Actual)')
gnb_wid_df9=gnb_widgets3_filter[['metric_name','acf2_id','slsbran','prodcd','salebal','franchise_cust',
                              'professional_cust','month_dt','gnb','auth_credit','gnb_volume',
                              'gnb_points','job_code','am_cost_center','region_id','record_date']]

## group by logon_id, metric_name, inthis
gnb_wid_df1=gnb_wid_df9.groupby(['am_cost_center','month_dt','metric_name','record_date'],as_index=False).aggregate({
                'gnb':'count'  # count widgets
                })

##transpose data
gnb_wid_df2 = gnb_wid_df1.pivot_table(index=['am_cost_center','metric_name','record_date'], columns=['month_dt'],
                     values='gnb', aggfunc='first', fill_value=0).reset_index()

#### get months
##rename columns **************CHANGE FOR FUTURE
cols = ['november','december','january','february','march','april','may','june','july',
        'august','september','october']
net_a = pd.DataFrame(list, index=['a',], columns=cols)


complementary = [c for c in net_a if c not in gnb_wid_df2]
gnb_wid_df3= pd.concat([net_a[complementary], gnb_wid_df2], axis=1, join='outer')

##delete the extra filler row
gnb_wid_df3= gnb_wid_df3.drop(labels='a', axis=0)
##convert to float
gnb_wid_df3 = gnb_wid_df3.astype({"november": float})
gnb_wid_df3 = gnb_wid_df3.astype({"december": float})
gnb_wid_df3 = gnb_wid_df3.astype({"january": float})
gnb_wid_df3 = gnb_wid_df3.astype({"february": float})
gnb_wid_df3 = gnb_wid_df3.astype({"march": float})
gnb_wid_df3 = gnb_wid_df3.astype({"april": float})
gnb_wid_df3 = gnb_wid_df3.astype({"may": float})
gnb_wid_df3 = gnb_wid_df3.astype({"june": float})
gnb_wid_df3 = gnb_wid_df3.astype({"july": float})
gnb_wid_df3 = gnb_wid_df3.astype({"august": float})
gnb_wid_df3 = gnb_wid_df3.astype({"september": float})
gnb_wid_df3 = gnb_wid_df3.astype({"october": float})

##order the vars
gnb_wid_ret=gnb_wid_df3[['metric_name','am_cost_center','november','december','january',
           'february','march','april','may','june','july','august','september','october','record_date']]


In [ ]:
################################################################################################
###################################### RETAIL GNB POINTS #######################################

df_gnb22=gnb_wid99[(gnb_wid99.gnb_points>0)]

##filter out the exclusion codes (including Professional Student LOC and Health LOC)
df_gnb33=df_gnb22[~df_gnb22['prodcd'].isin(professional_tuple)]

###group all data by points and month
df_gnb33['metric_name'] = df_gnb33['gnb_points']
df_gnb55= df_gnb33.astype({"metric_name": float})
gnb_imp44= df_gnb55.astype({"metric_name": str})

gnb_imp44["metric_name"].replace({"1.0": "GNB Points 1PT", "4.0": "GNB Points 4PTS","10.0": "GNB Points 10PTS","12.0": "GNB Points 12PTS"}, inplace=True)

gnb_imp11=gnb_imp44.groupby(['am_cost_center','month_dt','metric_name','record_date'],as_index=False).aggregate({
    'gnb_points':"sum"  # Sum points
    })

##transpose data
res = gnb_imp11.pivot_table(index=['am_cost_center','metric_name','record_date'], columns=['month_dt'],
                          values='gnb_points', aggfunc='first', fill_value=0).reset_index()


############################# create the total gnb points header ############################

result= gnb_imp44.groupby(by=['am_cost_center','month_dt','record_date'],as_index=False).aggregate({
    'gnb_points':"sum"  # Sum auth_credit volume
    })

result.rename(columns = {"gnb_points":'SBB TOTAL POINTS'}, inplace = True) 
result['metric_name']='Total GNB Points'


##transpose the data
res1 = result.pivot_table(index=['am_cost_center','metric_name','record_date'], columns=['month_dt'],
                          values='SBB TOTAL POINTS', aggfunc='first', fill_value=0).reset_index()

##merge to res
gnb_new1=pd.concat([res,res1],axis=0, ignore_index=True)

#### get months
##rename columns **************CHANGE FOR FUTURE
cols = ['november','december','january','february','march','april','may','june','july',
        'august','september','october']
net_a = pd.DataFrame(list, index=['a',], columns=cols)

complementary = [c for c in net_a if c not in gnb_new1]
gnb_wid_df5= pd.concat([net_a[complementary], gnb_new1], axis=1, join='outer')

##delete the extra filler row
gnb_wid_df5= gnb_wid_df5.drop(labels='a', axis=0)
##convert to float
gnb_wid_df5 = gnb_wid_df5.astype({"november": float})
gnb_wid_df5 = gnb_wid_df5.astype({"december": float})
gnb_wid_df5 = gnb_wid_df5.astype({"january": float})
gnb_wid_df5 = gnb_wid_df5.astype({"february": float})
gnb_wid_df5 = gnb_wid_df5.astype({"march": float})
gnb_wid_df5 = gnb_wid_df5.astype({"april": float})
gnb_wid_df5 = gnb_wid_df5.astype({"may": float})
gnb_wid_df5 = gnb_wid_df5.astype({"june": float})
gnb_wid_df5 = gnb_wid_df5.astype({"july": float})
gnb_wid_df5 = gnb_wid_df5.astype({"august": float})
gnb_wid_df5 = gnb_wid_df5.astype({"september": float})
gnb_wid_df5 = gnb_wid_df5.astype({"october": float})

##order the vars
gnb_pt_df6=gnb_wid_df5[['metric_name','am_cost_center','november','december','january',
           'february','march','april','may','june','july','august','september','october','record_date']]


In [ ]:
################################################################################################
############################ GNB RETAIL NET AUTHORIZED VOLUME ##################################

##TOTAL SBB NET AUTHORIZED VOLUME (SUM OF LON AND LOC)
total_net_auth_filter=gnb_wid99[gnb_wid99.prodcd.isin(gross_all_tuple)]
## create a variable to identify the metric
total_net_auth_filter=total_net_auth_filter.assign(metric_name='Total SBB Net Authorized Volume')
total_net_auth_filter=gnbfilterfunc(total_net_auth_filter)

#######LON
lon_filter=gnb_wid99[gnb_wid99.prodcd.isin(gross_lon_tuple)]
## create a variable to identify the metric
lon_filter=lon_filter.assign(metric_name='SBB Net Authorized LON Volume')
lon_filter=gnbfilterfunc(lon_filter)

#######LOC
loc_filter=gnb_wid99[gnb_wid99.prodcd.isin(gross_loc_tuple)]
## create a variable to identify the metric
loc_filter=loc_filter.assign(metric_name='SBB Net Authorized LOC Volume')
loc_filter=gnbfilterfunc(loc_filter)

#######SBB PROFESSIONAL BANKING
sbb_prof_filter = gnb_wid99[(gnb_wid99['professional_cust']=='Y') &
                           ((gnb_wid99.prodcd!='S021') | (gnb_wid99.prodcd!='S026') | (gnb_wid99.prodcd!='L290'))]
## create a variable to identify the metric
sbb_prof_filter=sbb_prof_filter.assign(metric_name='Net SBB Professional Banking')
sbb_prof_filter=gnbfilterfunc(sbb_prof_filter)

#######PROFESSIONAL STUDENT LOC
prof_sloc_filter=gnb_wid99[gnb_wid99.prodcd.isin(gross_student_loc_tuple)]
## create a variable to identify the metric
prof_sloc_filter=prof_sloc_filter.assign(metric_name='Net Professional Student LOC')
prof_sloc_filter=gnbfilterfunc(prof_sloc_filter)

#######FRANCHISE
franchise_filter = gnb_wid99[(gnb_wid99['franchise_cust']=='Y')]
## create a variable to identify the metric
franchise_filter=franchise_filter.assign(metric_name='Net Franchise Banking Net Volume')
franchise_filter=gnbfilterfunc(franchise_filter)


#######TOTAL PROFESSIONAL BANKING NET VOLUME (SUM OF SBB PROF BANKING, PROF STUDENT LOC, HEALTH)
total_prof_filter=pd.concat([sbb_prof_filter,prof_sloc_filter],axis=0)
total_prof_filter=total_prof_filter.drop(columns='metric_name')
## create a variable to identify the metric
total_prof_filter=total_prof_filter.assign(metric_name='Net Professional Banking Net Volume')
total_prof_filter=gnbfilterfunc(total_prof_filter)

########PROFESSIONAL AND FRANCHISE SUM
prof_and_franchise_filter=pd.concat([total_prof_filter,franchise_filter],axis=0)
prof_and_franchise_filter=prof_and_franchise_filter.drop(columns='metric_name')
## create a variable to identify the metric
prof_and_franchise_filter=prof_and_franchise_filter.assign(metric_name='Net Professional and Franchise Banking')
prof_and_franchise_filter=gnbfilterfunc(prof_and_franchise_filter)


## append all of the filters
new_df=pd.concat([total_net_auth_filter,lon_filter,loc_filter,sbb_prof_filter,prof_sloc_filter,
                  franchise_filter,total_prof_filter,prof_and_franchise_filter],axis=0)


In [ ]:
############## AGGREGATE DATA #######################

###group all data by LOGON_ID,MONTH,AND METRIC
    ## rollup to LOGON_ID
new_df1=new_df.groupby(['am_cost_center','month_dt','metric_name','record_date'],as_index=False).aggregate({
    'auth_credit':"sum"  # Sum auth_credit volume
    })

##transpose data
new_df2 = new_df1.pivot_table(index=['am_cost_center','metric_name','record_date'], columns=['month_dt'],
                     values='auth_credit', aggfunc='first', fill_value=0).reset_index()

#### get months
##rename columns **************CHANGE FOR FUTURE
cols = ['november','december','january','february','march','april','may','june','july',
        'august','september','october']
net_a = pd.DataFrame(list, index=['a',], columns=cols)


complementary = [c for c in net_a if c not in new_df2]
gnb_wid_df7= pd.concat([net_a[complementary], new_df2], axis=1, join='outer')

##delete the extra filler row
gnb_wid_df7= gnb_wid_df7.drop(labels='a', axis=0)
##convert to float
gnb_wid_df7 = gnb_wid_df7.astype({"november": float})
gnb_wid_df7 = gnb_wid_df7.astype({"december": float})
gnb_wid_df7 = gnb_wid_df7.astype({"january": float})
gnb_wid_df7 = gnb_wid_df7.astype({"february": float})
gnb_wid_df7 = gnb_wid_df7.astype({"march": float})
gnb_wid_df7 = gnb_wid_df7.astype({"april": float})
gnb_wid_df7 = gnb_wid_df7.astype({"may": float})
gnb_wid_df7 = gnb_wid_df7.astype({"june": float})
gnb_wid_df7 = gnb_wid_df7.astype({"july": float})
gnb_wid_df7 = gnb_wid_df7.astype({"august": float})
gnb_wid_df7 = gnb_wid_df7.astype({"september": float})
gnb_wid_df7 = gnb_wid_df7.astype({"october": float})

##order the vars
gnb_auth_df8=gnb_wid_df7[['metric_name','am_cost_center','november','december','january',
           'february','march','april','may','june','july','august','september','october','record_date']]


In [ ]:
####################### APPEND THE DATAFRAMES FOR WIDGETS,POINTS,VOLUME############################

gnb_all_retail=pd.concat([gnb_pt_df6,gnb_wid_ret,gnb_auth_df8],axis=0)


In [ ]:
######################## PROCESS RETAIL AM #######################################################

### calculate qtd and ytd
newvarfunc(gnb_all_retail)

## replace 0 with NaN so median only counts non-zeros
gnb_all_retail=gnb_all_retail.replace(0.0, np.nan)

### calculate qtd and ytd benchmarks
##CREATE A BENCHMARK (based on quarter median)
df_amsb_medianq1=gnb_all_retail.groupby(['metric_name'], as_index=False)['q1'].median()
df_amsb_medianq2=gnb_all_retail.groupby(['metric_name'], as_index=False)['q2'].median()
df_amsb_medianq3=gnb_all_retail.groupby(['metric_name'], as_index=False)['q3'].median()
df_amsb_medianq4=gnb_all_retail.groupby(['metric_name'], as_index=False)['q4'].median()
df_amsb_medianytd=gnb_all_retail.groupby(['metric_name'], as_index=False)['ytd'].median()
##rename median
df_amsb_medianq1.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
df_amsb_medianq2.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
df_amsb_medianq3.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
df_amsb_medianq4.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
df_amsb_medianytd.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)

### merge to dataframe
gnb_pt1=pd.merge(pd.merge(gnb_all_retail,df_amsb_medianq1,
                on=['metric_name'],
                how='left'),df_amsb_medianq2,on=['metric_name'],how='left')

gnb_pt2=pd.merge(pd.merge(gnb_pt1,df_amsb_medianq3,
                on=['metric_name'],
                how='left'),df_amsb_medianq4,on=['metric_name'],how='left')

gnb_pt3=pd.merge(gnb_pt2,df_amsb_medianytd,on=['metric_name'],how="left")


############## merge to hr/alignment to get am/sm cc
df = spark.table('anp_cabbtdct1_final.latest_hrm_sbb')
dfhr=df.toPandas()

dfhr=dfhr[['am_cost_center','am_cost_center_name','am_cost_center_full_name','sm_cost_center','sm_cost_center_name']].drop_duplicates()

## merge 
gnb_all=pd.merge(gnb_pt3,dfhr,how='left',left_on=['am_cost_center'],right_on=['am_cost_center'])

gnb_all['level']='AM'
gnb_all['scorecard_filter']='Retail'
gnb_all['acf2_id']=''
gnb_all['employee_name']=gnb_all.am_cost_center_full_name

gnb_am_retail_all=gnb_all[['metric_name','level','acf2_id','employee_name','november','december','january',
           'february','march','april','may','june','july','august','september','october',
           'q1','q1_benchmark','q2','q2_benchmark','q3','q3_benchmark','q4','q4_benchmark',
           'ytd','ytd_benchmark','am_cost_center','am_cost_center_name',
           'am_cost_center_full_name','sm_cost_center','sm_cost_center_name',
           'scorecard_filter','record_date']]


In [ ]:
############################# CREATE THE SM RETAIL OUTPUT ##########################################

gnb111=smgrpfunc(gnb_am_retail_all)

##CREATE THE Q1,Q2,Q3,Q3,YTD VARIABLES
newvarfunc(gnb111)

## replace 0 with NaN so median only counts non-zeros
gnb111=gnb111.replace(0.0, np.nan)

### calculate qtd and ytd benchmarks
##CREATE A BENCHMARK (based on quarter median)
df_sm_medianq1=gnb111.groupby(['metric_name'], as_index=False)['q1'].median()
df_sm_medianq2=gnb111.groupby(['metric_name'], as_index=False)['q2'].median()
df_sm_medianq3=gnb111.groupby(['metric_name'], as_index=False)['q3'].median()
df_sm_medianq4=gnb111.groupby(['metric_name'], as_index=False)['q4'].median()
df_sm_medianytd=gnb111.groupby(['metric_name'], as_index=False)['ytd'].median()
##rename median
df_sm_medianq1.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
df_sm_medianq2.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
df_sm_medianq3.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
df_sm_medianq4.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
df_sm_medianytd.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)

### merge to dataframe
gnb_pt111=pd.merge(pd.merge(gnb111,df_sm_medianq1,
                on=['metric_name'],
                how='left'),df_sm_medianq2,on=['metric_name'],how='left')

gnb_pt222=pd.merge(pd.merge(gnb_pt111,df_sm_medianq3,
                on=['metric_name'],
                how='left'),df_sm_medianq4,on=['metric_name'],how='left')

gnb_pt333=pd.merge(gnb_pt222,df_sm_medianytd,on=['metric_name'],how="left")

gnb_pt333['acf2_id']=''
gnb_pt333['level']='SM'
gnb_pt333['scorecard_filter']='Retail'
gnb_pt333['employee_name']=gnb_pt333.sm_cost_center_name
gnb_pt333['am_cost_center']=''
gnb_pt333['am_cost_center_name']=''
gnb_pt333['am_cost_center_full_name']=''

gnb_pt344=gnb_pt333[['metric_name','level','acf2_id','employee_name','november','december','january',
           'february','march','april','may','june','july','august','september','october',
           'q1','q1_benchmark','q2','q2_benchmark','q3','q3_benchmark','q4','q4_benchmark',
           'ytd','ytd_benchmark','am_cost_center','am_cost_center_name',
           'am_cost_center_full_name','sm_cost_center','sm_cost_center_name',
           'scorecard_filter','record_date']]


In [ ]:
############################# CREATE THE AM TOTAL OUTPUT ##########################################
## concatenate sbb and retail filters
gnb_am=pd.concat([gnb_am_retail_all,gnb_am_sbb],axis=0)

## group by AM
gnb110=amgrpfunc(gnb_am)

##CREATE THE Q1,Q2,Q3,Q3,YTD VARIABLES
newvarfunc(gnb110)

## replace 0 with NaN so median only counts non-zeros
gnb110=gnb110.replace(0.0, np.nan)

### calculate qtd and ytd benchmarks
##CREATE A BENCHMARK (based on quarter median)
df_am_medianq1=gnb110.groupby(['metric_name'], as_index=False)['q1'].median()
df_am_medianq2=gnb110.groupby(['metric_name'], as_index=False)['q2'].median()
df_am_medianq3=gnb110.groupby(['metric_name'], as_index=False)['q3'].median()
df_am_medianq4=gnb110.groupby(['metric_name'], as_index=False)['q4'].median()
df_am_medianytd=gnb110.groupby(['metric_name'], as_index=False)['ytd'].median()
##rename median
df_am_medianq1.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
df_am_medianq2.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
df_am_medianq3.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
df_am_medianq4.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
df_am_medianytd.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)

### merge to dataframe
gnb_pt11=pd.merge(pd.merge(gnb110,df_am_medianq1,
                on=['metric_name'],
                how='left'),df_am_medianq2,on=['metric_name'],how='left')

gnb_pt22=pd.merge(pd.merge(gnb_pt11,df_am_medianq3,
                on=['metric_name'],
                how='left'),df_am_medianq4,on=['metric_name'],how='left')

gnb_pt338=pd.merge(gnb_pt22,df_am_medianytd,on=['metric_name'],how="left")

gnb_pt338['acf2_id']=''
gnb_pt338['level']='AM'
gnb_pt338['scorecard_filter']='Total'
gnb_pt338['employee_name']=gnb_pt338.am_cost_center_full_name

gnb_pt348=gnb_pt338[['metric_name','level','acf2_id','employee_name','november','december','january',
           'february','march','april','may','june','july','august','september','october',
           'q1','q1_benchmark','q2','q2_benchmark','q3','q3_benchmark','q4','q4_benchmark',
           'ytd','ytd_benchmark','am_cost_center','am_cost_center_name',
           'am_cost_center_full_name','sm_cost_center','sm_cost_center_name',
           'scorecard_filter','record_date']]


In [ ]:
############################# CREATE THE SM TOTAL OUTPUT ##########################################

## concatenate sbb and retail filters
gnb_sm=pd.concat([gnb_pt344,gnb_sm_sbb],axis=0)

gnb111=smgrpfunc(gnb_pt348)

##CREATE THE Q1,Q2,Q3,Q3,YTD VARIABLES
newvarfunc(gnb111)

## replace 0 with NaN so median only counts non-zeros
gnb111=gnb111.replace(0.0, np.nan)

### calculate qtd and ytd benchmarks
##CREATE A BENCHMARK (based on quarter median)
df_sm_medianq1=gnb111.groupby(['metric_name'], as_index=False)['q1'].median()
df_sm_medianq2=gnb111.groupby(['metric_name'], as_index=False)['q2'].median()
df_sm_medianq3=gnb111.groupby(['metric_name'], as_index=False)['q3'].median()
df_sm_medianq4=gnb111.groupby(['metric_name'], as_index=False)['q4'].median()
df_sm_medianytd=gnb111.groupby(['metric_name'], as_index=False)['ytd'].median()
##rename median
df_sm_medianq1.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
df_sm_medianq2.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
df_sm_medianq3.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
df_sm_medianq4.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
df_sm_medianytd.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)

### merge to dataframe
gnb_pt111=pd.merge(pd.merge(gnb111,df_sm_medianq1,
                on=['metric_name'],
                how='left'),df_sm_medianq2,on=['metric_name'],how='left')

gnb_pt222=pd.merge(pd.merge(gnb_pt111,df_sm_medianq3,
                on=['metric_name'],
                how='left'),df_sm_medianq4,on=['metric_name'],how='left')

gnb_pt30=pd.merge(gnb_pt222,df_sm_medianytd,on=['metric_name'],how="left")

gnb_pt30['acf2_id']=''
gnb_pt30['level']='SM'
gnb_pt30['scorecard_filter']='Total'
gnb_pt30['employee_name']=gnb_pt30.sm_cost_center_name
gnb_pt30['am_cost_center']=''
gnb_pt30['am_cost_center_name']=''
gnb_pt30['am_cost_center_full_name']=''

gnb_pt350=gnb_pt30[['metric_name','level','acf2_id','employee_name','november','december','january',
           'february','march','april','may','june','july','august','september','october',
           'q1','q1_benchmark','q2','q2_benchmark','q3','q3_benchmark','q4','q4_benchmark',
           'ytd','ytd_benchmark','am_cost_center','am_cost_center_name',
           'am_cost_center_full_name','sm_cost_center','sm_cost_center_name',
           'scorecard_filter','record_date']]


In [ ]:
################################# CREATE THE NATIONAL OFFICE TOTAL #############################

gnb115=natgrpfunc(gnb_pt350)

##CREATE THE Q1,Q2,Q3,Q3,YTD VARIABLES
newvarfunc(gnb115)

## replace 0 with NaN so median only counts non-zeros
gnb115=gnb115.replace(0.0, np.nan)

### calculate qtd and ytd benchmarks
##CREATE A BENCHMARK (based on quarter median)
df_nso_medianq1=gnb115.groupby(['metric_name'], as_index=False)['q1'].median()
df_nso_medianq2=gnb115.groupby(['metric_name'], as_index=False)['q2'].median()
df_nso_medianq3=gnb115.groupby(['metric_name'], as_index=False)['q3'].median()
df_nso_medianq4=gnb115.groupby(['metric_name'], as_index=False)['q4'].median()
df_nso_medianytd=gnb115.groupby(['metric_name'], as_index=False)['ytd'].median()
##rename median
df_nso_medianq1.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
df_nso_medianq2.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
df_nso_medianq3.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
df_nso_medianq4.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
df_nso_medianytd.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)

### merge to dataframe
gnb_pt116=pd.merge(pd.merge(gnb115,df_nso_medianq1,
                on=['metric_name'],
                how='left'),df_nso_medianq2,on=['metric_name'],how='left')

gnb_pt225=pd.merge(pd.merge(gnb_pt116,df_nso_medianq3,
                on=['metric_name'],
                how='left'),df_nso_medianq4,on=['metric_name'],how='left')

gnb_pt300=pd.merge(gnb_pt225,df_nso_medianytd,on=['metric_name'],how="left")

gnb_pt300['acf2_id']=''
gnb_pt300['level']='National Office'
gnb_pt300['scorecard_filter']='Total'
gnb_pt300['employee_name']='National Office'
gnb_pt300['am_cost_center']=''
gnb_pt300['am_cost_center_name']=''
gnb_pt300['am_cost_center_full_name']=''
gnb_pt300['sm_cost_center']=''
gnb_pt300['sm_cost_center_name']=''

gnb_pt300['record_date']=rec_date

gnb_ptnso=gnb_pt300[['metric_name','level','acf2_id','employee_name','november','december','january',
           'february','march','april','may','june','july','august','september','october',
           'q1','q1_benchmark','q2','q2_benchmark','q3','q3_benchmark','q4','q4_benchmark',
           'ytd','ytd_benchmark','am_cost_center','am_cost_center_name',
           'am_cost_center_full_name','sm_cost_center','sm_cost_center_name',
           'scorecard_filter','record_date']]


In [ ]:
#################################### CONCATENATE ALL DATAFRAMES ################################

## amsb,ramsb, am sbb, sm sbb, am retail, sm retail, am total, sm total, nso total
gnb_allf=pd.concat([gnb_amsb,gnb_sm_sbb,gnb_am_sbb,
                  gnb_am_retail_all,gnb_pt344,
                  gnb_pt348,gnb_pt350,gnb_ptnso],axis=0)


In [ ]:
gnb_allf.head(4)

In [ ]:
gnb_allf.info()

In [ ]:
## fill in with zeroes
gnb_allf=gnb_allf.replace([np.inf, -np.inf], np.nan)

gnb_allf[['november','december','january','february','march','april','may','june','july','august',
                  'september','october','q1','q1_benchmark','q2','q2_benchmark','q3','q3_benchmark',
                            'q4','q4_benchmark','ytd','ytd_benchmark']]=gnb_allf[['november','december','january','february','march','april','may','june','july','august',
                  'september','october','q1','q1_benchmark','q2','q2_benchmark','q3','q3_benchmark',
                            'q4','q4_benchmark','ytd','ytd_benchmark']].fillna(value=0)


In [ ]:
#from pyspark.sql.types import *
#create schema for your dataframe
#schema = StructType([StructField("metric_name", StringType(), True)\
#                   ,StructField("level", StringType(), True)\
#                   ,StructField("acf2_id", StringType(), True)\
#                     ,StructField("employee_name", StringType(), True)\
#                     ,StructField("november", FloatType(), True)\
#                     ,StructField("december", FloatType(), True)\
#                     ,StructField("january", FloatType(), True)\
#                     ,StructField("february", FloatType(), True)\
#                     ,StructField("march", FloatType(), True)\
#                     ,StructField("april", FloatType(), True)\
#                     ,StructField("may", FloatType(), True)\
#                     ,StructField("june", FloatType(), True)\
#                     ,StructField("july", FloatType(), True)\
#                     ,StructField("august", FloatType(), True)\
#                     ,StructField("september", FloatType(), True)\
#                     ,StructField("october", FloatType(), True)\
#                     ,StructField("q1", FloatType(), True)\
#                     ,StructField("q1_benchmark", FloatType(), True)\
#                     ,StructField("q2", FloatType(), True)\
#                     ,StructField("q2_benchmark", FloatType(), True)\
#                     ,StructField("q3", FloatType(), True)\
#                     ,StructField("q3_benchmark", FloatType(), True)\
#                    ,StructField("q4", FloatType(), True)\
#                     ,StructField("q4_benchmark", FloatType(), True)\
#                     ,StructField("ytd", FloatType(), True)\
#                     ,StructField("ytd_benchmark", FloatType(), True)\
#                     ,StructField("am_cost_center", StringType(), True)\
#                     ,StructField("am_cost_center_name", StringType(), True)\
#                     ,StructField("am_cost_center_full_name", StringType(), True)\
#                     ,StructField("sm_cost_center", StringType(), True)\
#                      ,StructField("sm_cost_center_name", StringType(), True)\
#                      ,StructField("scorecard_filter", StringType(), True)\
#                      ,StructField("record_date", StringType(), True)])

#create spark dataframe using schema
#gnb_allfinal = spark.createDataFrame(gnb_allf,schema=schema)

In [ ]:
## save in table
spark.sql("set hive.exec.dynamic.partition.mode=nonstrict")
spark.createDataFrame(gnb_allf).write.mode("append").partitionBy("record_date").format("hive").saveAsTable("anp_cabbtdct1_final.SBB_SC_OUTPUT_GNB")


In [ ]:
## END OF PROGRAM